In [21]:
import json
import re
import os
import pandas as pd

from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary
import pyLDAvis.gensim

import nltk
from nltk.stem.wordnet import WordNetLemmatizer
stopwords = set(nltk.corpus.stopwords.words('english'))

data_path = '../data/'
model_path = '../model/'

def load_webhouse_data(data_name): 
    data_file = data_path + data_name + '.json'
    with open(data_file) as json_data:
        data = json.load(json_data)
        return data

def cleanup_text(text):
    text = re.sub(r'http\S+', '', text)
    text = text.replace("'s", " ")
    text = text.replace("n't", " not ")
    text = text.replace("'ve", " have ")
    text = text.replace("'re", " are ")
    text = text.replace("I'm"," I am ")
    text = text.replace("you're"," you are ")
    text = text.replace("You're"," You are ")
    text = text.replace("-"," ")
    text = text.replace("/"," ")
    text = text.replace("("," ")
    text = text.replace(")"," ")
    text = text.replace("%"," percent ")
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    text = " ".join([i for i in text.lower().split() if i not in stopwords])
    token = [WordNetLemmatizer().lemmatize(i) for i in text.split()]
    return token
    
# data = load_webhouse_data('microsoft')
data = load_webhouse_data('microsoft_0419_0519_clean') 

In [ ]:
titles = [[data[i]['title'], data[i]['published'][:10]]  for i in range(len(data))]
df_feeds = pd.DataFrame(titles,columns=['title', 'date'])
titles = df_feeds[['title']].applymap(cleanup_text)['title']

In [ ]:
dictionary = Dictionary(titles)
dictionary.filter_extremes(no_below=10, no_above=0.8)
corpora = [dictionary.doc2bow(doc) for doc in titles]

# Running and Trainign LDA model on the document term matrix.
lda_model = ldamodel.LdaModel(corpora, num_topics=8, id2word = dictionary, passes=50)